In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from loguru import logger
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

processed = Path("../data/processed")
datafile = processed / "whatsapp-20240122-182706.parq"
datafile = processed / "whatsapp-20240122-222233.parq"
if not datafile.exists():
    logger.warning("Datafile does not exist. First run src/preprocess.py, and check the timestamp!")

In [ ]:
df = pd.read_parquet(datafile)
df

Let's extract some more info from the timestamp:

In [ ]:
df["date"] = df["timestamp"].dt.date
df["isoweek"] = df["timestamp"].dt.isocalendar().week
df.head()


Now, we can group by the isoweeks, for example. 
Let's reindex in order to fill the missing weeks.

In [ ]:
p = df.groupby("isoweek").count()
p.head()

In [ ]:
# lets reindex the isoweeks. This wont work if you have multiple years!
p = p.reindex(range(int(p.index.min()), int(p.index.max()) + 1), fill_value=0)
sns.scatterplot(data=p, x="isoweek", y="timestamp")
sns.lineplot(data=p, x="isoweek", y="timestamp")

Try to play with the colors. Make sure adding colors conveys a message, and isnt randomly adding colors.

In [ ]:
p = df.groupby(["author", "isoweek"]).count()
sns.scatterplot(data=p, x="isoweek", y="timestamp", hue="author", legend=False)
sns.lineplot(data=p, x="isoweek", y="timestamp", legend=False)

In [ ]:
df = pd.read_parquet(datafile)
df["date"] = df["timestamp"].dt.date
df["day_of_week"] = df["timestamp"].dt.day_name()
df_agg = df.groupby(['date', 'day_of_week']).size().reset_index(name='message_count')
df_agg

In [ ]:

# Set the order of the days of the week
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Create a FacetGrid to plot the line plots for each day of the week
g = sns.FacetGrid(df_agg, col='day_of_week', col_wrap=4, height=3, col_order=days_order)
g.map(sns.scatterplot, 'date', 'message_count')
g.map(sns.lineplot, 'date', 'message_count')
g.set(xticks=[])
g.despine(left=True, bottom=False)

In [ ]:
df.head()

In [ ]:
import plotly.express as px
# first count the number for every category, for every date
p = df.groupby(["date", "timestamp_category"]).size().reset_index(name='message_count')
# then, make the count cumulative for every category
p['cumulative_count'] = p.groupby('timestamp_category')['message_count'].cumsum()

fig = px.area(p, x="date", y="cumulative_count", color="timestamp_category",
              line_group="timestamp_category", labels={"cumulative_count": "Cumulative Message Count"})
fig.show()